## Accelerate Inference: Neural Network Pruning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pickle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers
from tensorflow.keras.layers import *

print(tf.version.VERSION)

2.9.2


In [124]:
# untar
!tar -xvzf dataset.tar.gz
# load train
train_images = pickle.load(open('train_images.pkl', 'rb'))
train_labels = pickle.load(open('train_labels.pkl', 'rb'))
# load val
val_images = pickle.load(open('val_images.pkl', 'rb'))
val_labels = pickle.load(open('val_labels.pkl', 'rb'))

train_images.pkl
train_labels.pkl
val_images.pkl
val_labels.pkl


In [133]:
# Define the neural network architecture (don't change this)

model = models.Sequential()
model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))

In [134]:
print(model.summary())

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_48 (Conv2D)          (None, 25, 25, 32)        896       
                                                                 
 activation_78 (Activation)  (None, 25, 25, 32)        0         
                                                                 
 conv2d_49 (Conv2D)          (None, 23, 23, 32)        9248      
                                                                 
 activation_79 (Activation)  (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 11, 11, 32)       0         
 g2D)                                                            
                                                                 
 dropout_39 (Dropout)        (None, 11, 11, 32)        0         
                                                     

In [135]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs = 50
history = model.fit(train_images, train_labels, batch_size=32, epochs=epochs, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

Epoch 1/50
703/703 [==============================] - 4s 5ms/step - loss: 1.5304 - accuracy: 0.3023 - val_loss: 1.4099 - val_accuracy: 0.3735
Epoch 2/50
703/703 [==============================] - 4s 5ms/step - loss: 1.3552 - accuracy: 0.4141 - val_loss: 1.2658 - val_accuracy: 0.4677
Epoch 3/50
703/703 [==============================] - 3s 5ms/step - loss: 1.2923 - accuracy: 0.4490 - val_loss: 1.2163 - val_accuracy: 0.4859
Epoch 4/50
703/703 [==============================] - 3s 5ms/step - loss: 1.2386 - accuracy: 0.4848 - val_loss: 1.1732 - val_accuracy: 0.5073
Epoch 5/50
703/703 [==============================] - 3s 5ms/step - loss: 1.2057 - accuracy: 0.5020 - val_loss: 1.1384 - val_accuracy: 0.5251
Epoch 6/50
703/703 [==============================] - 4s 5ms/step - loss: 1.1728 - accuracy: 0.5229 - val_loss: 1.1192 - val_accuracy: 0.5450
Epoch 7/50
703/703 [==============================] - 3s 5ms/step - loss: 1.1415 - accuracy: 0.5389 - val_loss: 1.0809 - val_accuracy: 0.5517
Epoch 

In [136]:
results = model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 0s 5ms/step - loss: 0.7461 - accuracy: 0.7204


In [137]:
# perform pruning here

# get the weights 
weights = model.get_weights()

# you can use set_weights() to set some weights to zero, e.g.,
# some references for pruning techniques: https://arxiv.org/pdf/1810.05270v2.pdf, https://arxiv.org/pdf/2001.04062.pdf

# get the layers
layers = model.layers

from keras import backend as K
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

prune_rate = 30
def compute_mask(weights):
  shape = weights.shape
  abs_weights = np.abs(weights)
  # abs_weights = abs_weights.reshape((-1, shape[-1]))
  abs_weights = abs_weights.reshape(-1)
  threshold = np.percentile(abs_weights, prune_rate)
  # threshold = tfp.stats.percentile(abs_weights, prune_rate)
  abs_weights = abs_weights.reshape(shape)
  # abs_weights = K.eval(abs_weights)
  
  mask = np.zeros(shape)
  mask[abs_weights < threshold] = 1
  # mask = mask.reshape(shape)
  return mask

constant = 0 # (will annulate kernel, but not bias) 
def stop_backprop(x, layers, i):
  layer = layers[i]
  weights0, weights1 = layer.weights

  mask = compute_mask(weights0)
  weights0 = weights0 * (1 - mask) + constant * mask

  # stopped = K.stop_gradient(weights0)
  # weights0 = weights0 * (1 - mask) + stopped * mask

  layer.set_weights([weights0, weights1])
  return layer(x)
    
pruned_model = models.Sequential()
for i, layer in enumerate(layers):
  # if len(layer.weights) > 0 and 'conv2d' in layer.name:
  if len(layer.weights) > 0:
    # print(len(layer.weights))
    # print(layer.weights[0].shape)
    # print(layer.weights[1].shape)
    
    new_layer = Lambda(stop_backprop, arguments={'layers': layers, 'i': i})
    # new_layer.set_weights(layer.weights)
    pruned_model.add(new_layer)
  else:
    pruned_model.add(layer)



In [138]:
# you can use the default hyper-parameters for training, 
# and val accuracy ~59% after 25 epochs and > 63% after 50 epochs

pruned_model.compile(run_eagerly=True, optimizer=keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

epochs = 1
history = pruned_model.fit(train_images, train_labels, batch_size=32, epochs=epochs, 
                    validation_data=(val_images, val_labels)) # train for 50 epochs, with batch size 32

The following Variables were used a Lambda layer's call (lambda_126), but
are not present in its tracked objects:
  <tf.Variable 'conv2d_48/kernel:0' shape=(3, 3, 3, 32) dtype=float32, numpy=
array([[[[-0.09391245, -0.12319858,  0.0630223 ,  0.05146995,
           0.09712143, -0.06017947,  0.        , -0.1338746 ,
           0.13758533,  0.        ,  0.12997559, -0.14218195,
           0.        , -0.05573569, -0.06492909,  0.        ,
           0.        ,  0.09718483,  0.        , -0.10073642,
          -0.12685847,  0.        , -0.11324305, -0.07133701,
           0.        ,  0.08169644,  0.09012291,  0.13352159,
          -0.04998351,  0.05446361,  0.        , -0.14893429],
         [-0.14182726, -0.09173274, -0.11326265,  0.10635749,
           0.05046377,  0.        ,  0.        ,  0.        ,
           0.06524117,  0.07906832,  0.        , -0.10011062,
          -0.08106556, -0.0489588 ,  0.        ,  0.14082575,
           0.1104186 ,  0.        ,  0.        ,  0.1030073 ,
 

703/703 [==============================] - 47s 67ms/step - loss: 0.8742 - accuracy: 0.6682 - val_loss: 0.9359 - val_accuracy: 0.6467


In [139]:
print(pruned_model.summary())

Model: "sequential_35"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda_126 (Lambda)         multiple                  0         
                                                                 
 activation_78 (Activation)  (None, 25, 25, 32)        0         
                                                                 
 lambda_127 (Lambda)         multiple                  0         
                                                                 
 activation_79 (Activation)  (None, 23, 23, 32)        0         
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 11, 11, 32)       0         
 g2D)                                                            
                                                                 
 dropout_39 (Dropout)        (None, 11, 11, 32)        0         
                                                     

In [122]:
# you need to save the model's weights, naming it 'my_model_weights.h5'
model.save_weights("my_model_weights_1.h5")

# running this cell will immediately download a file called 'my_model_weights.h5'
from google.colab import files
files.download("my_model_weights_1.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [140]:
results = pruned_model.evaluate(val_images, val_labels, batch_size=128)

20/20 [==============================] - 1s 56ms/step - loss: 0.9359 - accuracy: 0.6467


In [141]:
# perform pruning here

# get the weights 
# weights = pruned_model.get_weights()
weights = model.get_weights()

# you can use set_weights() to set some weights to zero, e.g.,
# some references for pruning techniques: https://arxiv.org/pdf/1810.05270v2.pdf, https://arxiv.org/pdf/2001.04062.pdf

# model.set_weights(weights)
# print(weights)

global zero_nums
zero_nums = 0
def compute(arr):
  if hasattr(arr, '__len__'):
    for i in arr:
      compute(i)
  else:
    # print(type(arr))
    if arr == 0:
      global zero_nums
      zero_nums += 1


nums = 592933
compute(weights)
# # zero_nums = 0
# for w in weights:
#   # nums += len(w)
#   for i in w:
#     if i == 0:
#       zero_nums += 1

print(zero_nums / nums)


0.29964262404015296


In [14]:
# Define the neural network architecture (don't change this)
class CustomizedConv2D(Conv2D):
  def __init__(self, *args, **kwargs):
    super(CustomizedConv2D, self).__init__(*args, **kwargs)

model = models.Sequential()
# model.add(Conv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(CustomizedConv2D(32, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5), input_shape=(25,25,3)))
model.add(Activation('relu'))
# model.add(Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(CustomizedConv2D(32, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
# model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(CustomizedConv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
# model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(CustomizedConv2D(64, (3, 3), kernel_regularizer=regularizers.l2(1e-5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))